In [ ]:
# ==================== 导入必要的库 ====================
# %matplotlib inline: 在Jupyter中内嵌显示matplotlib图像
%matplotlib inline

# torch: PyTorch深度学习框架
import torch

# torchvision: PyTorch的计算机视觉库，包含预训练模型和数据集
import torchvision

# nn: PyTorch的神经网络模块
from torch import nn

# F: PyTorch的函数式接口，包含各种操作函数
from torch.nn import functional as F

# d2l: Dive into Deep Learning工具库
from d2l import torch as d2l

In [ ]:
# ==================== 加载预训练的ResNet-18模型 ====================

# 加载在ImageNet上预训练的ResNet-18模型
# 
# 为什么使用预训练模型？
# 1. 迁移学习：利用在大数据集上学到的特征
# 2. 加速训练：不需要从头训练
# 3. 更好的性能：预训练权重提供了良好的初始化
pretrained_net = torchvision.models.resnet18(pretrained=True)

# 查看ResNet-18的最后三层
# children()返回模型的直接子模块列表
# ResNet最后通常是：平均池化层、展平层、全连接层
print(list(pretrained_net.children())[-3:])

In [ ]:
# ==================== 构建FCN的特征提取部分 ====================

# 去掉ResNet-18的最后两层（全局平均池化和全连接层）
# 
# 为什么要去掉？
# 1. 全连接层会丢失空间信息
# 2. 语义分割需要保留空间位置信息
# 3. FCN的核心思想：全卷积网络，不使用全连接层
# 
# [:-2] 保留除最后两层外的所有层
# *list(...) 将列表展开为参数
# nn.Sequential 将这些层组合成一个顺序模型
net = nn.Sequential(*list(pretrained_net.children())[:-2])

# 测试网络输出形状
# 输入：1张图片，3通道(RGB)，320x480的分辨率
X = torch.randn(1, 3, 320, 480)

# 输出：特征图的形状
# 经过ResNet卷积后，空间尺寸缩小，通道数增加到512
# 预期输出形状：(1, 512, 10, 15)
# 高度：320/32=10，宽度：480/32=15（ResNet下采样32倍）
print(net(X).shape)

In [ ]:
# ==================== 添加FCN的分类和上采样层 ====================

# Pascal VOC数据集包含21个类别
# 0: 背景，1-20: 各种物体类别（飞机、自行车、鸟等）
num_classes = 21

# 添加1x1卷积层：将512个通道转换为21个类别通道
# 
# 1x1卷积的作用：
# 1. 降维：从512通道降到21通道
# 2. 分类：每个通道对应一个类别的预测
# 3. 保持空间信息：不改变特征图的高度和宽度
net.add_module('final_conv', nn.Conv2d(512, num_classes, kernel_size=1))

# 添加转置卷积层：将特征图上采样回原始图像大小
# 
# 参数解释：
# - in_channels=num_classes: 输入21个类别通道
# - out_channels=num_classes: 输出21个类别通道
# - kernel_size=64: 卷积核大小（较大的核用于平滑上采样）
# - padding=16: 填充，用于调整输出尺寸
# - stride=32: 步幅32，将特征图放大32倍（恢复到原始尺寸）
# 
# 为什么stride=32？
# 因为ResNet将图像下采样了32倍，现在需要上采样32倍恢复
net.add_module('transpose_conv', nn.ConvTranspose2d(num_classes, num_classes,
                                               kernel_size=64, padding=16,
                                               stride=32))

In [ ]:
# ==================== 双线性插值卷积核 ====================

def bilinear_kernel(in_channels, out_channels, kernel_size):
    """
    构造用于双线性插值的转置卷积核
    
    为什么需要双线性插值？
    - 转置卷积的权重需要初始化
    - 双线性插值是一种平滑的上采样方法
    - 它比随机初始化的权重效果更好，训练更稳定
    
    双线性插值原理：
    - 距离中心越近的像素权重越大
    - 距离中心越远的像素权重越小
    - 这样可以产生平滑的放大效果
    
    参数:
        in_channels: 输入通道数
        out_channels: 输出通道数
        kernel_size: 卷积核大小
    
    返回:
        weight: 初始化好的卷积核权重
    """
    # 计算中心位置的因子
    # factor用于计算权重衰减的速度
    factor = (kernel_size + 1) // 2
    
    # 确定卷积核的中心位置
    if kernel_size % 2 == 1:
        # 奇数大小：中心是整数位置
        center = factor - 1
    else:
        # 偶数大小：中心是0.5的位置
        center = factor - 0.5
    
    # 创建网格坐标
    # og[0]: 行坐标网格 (kernel_size, 1)
    # og[1]: 列坐标网格 (1, kernel_size)
    og = (torch.arange(kernel_size).reshape(-1, 1),
          torch.arange(kernel_size).reshape(1, -1))
    
    # 计算双线性插值的权重
    # 公式：(1 - |x-center|/factor) * (1 - |y-center|/factor)
    # 离中心越近，权重越大；离中心越远，权重越小
    filt = (1 - torch.abs(og[0] - center) / factor) * \
           (1 - torch.abs(og[1] - center) / factor)
    
    # 初始化权重张量
    # 形状：(in_channels, out_channels, kernel_size, kernel_size)
    weight = torch.zeros((in_channels, out_channels,
                          kernel_size, kernel_size))
    
    # 只为对角线位置（输入通道i对应输出通道i）赋值
    # 这保证了每个通道独立处理，不会混合颜色通道
    weight[range(in_channels), range(out_channels), :, :] = filt
    
    return weight

In [ ]:
# ==================== 测试双线性插值上采样效果 ====================

# 创建一个转置卷积层用于测试
# 3个输入通道(RGB)，3个输出通道，卷积核4x4
# padding=1, stride=2: 将图像放大2倍
# bias=False: 不使用偏置项
conv_trans = nn.ConvTranspose2d(3, 3, kernel_size=4, padding=1, stride=2,
                                bias=False)

# 用双线性插值核初始化转置卷积的权重
conv_trans.weight.data.copy_(bilinear_kernel(3, 3, 4))

# 读取测试图像并转换为张量
# ToTensor()会将PIL图像转换为形状(C, H, W)的张量，值域[0,1]
img = torchvision.transforms.ToTensor()(d2l.Image.open('./catdog.jpg'))

# 添加批次维度：(C, H, W) -> (1, C, H, W)
X = img.unsqueeze(0)

# 执行转置卷积，放大图像
Y = conv_trans(X)

# 移除批次维度并调整通道顺序以便显示
# (1, C, H, W) -> (C, H, W) -> (H, W, C)
out_img = Y[0].permute(1, 2, 0).detach()

# 设置图像显示大小
d2l.set_figsize()

# 显示原始图像
print('input image shape:', img.permute(1, 2, 0).shape)
d2l.plt.imshow(img.permute(1, 2, 0))

# 显示放大后的图像（应该是原始图像的2倍大小）
print('output image shape:', out_img.shape)
d2l.plt.imshow(out_img)

In [ ]:
# ==================== 初始化FCN的转置卷积层 ====================

# 使用双线性插值核初始化FCN网络中的转置卷积层
# 
# 参数说明：
# - num_classes: 21个类别，输入和输出通道都是21
# - 64: 卷积核大小
# 
# 这样初始化的好处：
# 1. 提供了良好的起点，而不是随机权重
# 2. 保证了平滑的上采样效果
# 3. 加速训练收敛
W = bilinear_kernel(num_classes, num_classes, 64)
net.transpose_conv.weight.data.copy_(W)

In [ ]:
# ==================== 加载VOC数据集 ====================

# 设置批次大小和裁剪尺寸
batch_size = 32           # 每批处理32张图像
crop_size = (320, 480)    # 将图像裁剪为320x480

# 加载VOC语义分割数据集
# train_iter: 训练集数据迭代器
# test_iter: 测试集数据迭代器
# 
# 这个函数会：
# 1. 下载VOC2012数据集（如果还没下载）
# 2. 创建数据集对象
# 3. 创建数据加载器，支持批处理和多进程加载
train_iter, test_iter = d2l.load_data_voc(batch_size, crop_size)

In [ ]:
# ==================== 定义损失函数并开始训练 ====================

def loss(inputs, targets):
    """
    语义分割的损失函数
    
    参数:
        inputs: 模型预测，形状(batch_size, num_classes, H, W)
        targets: 真实标签，形状(batch_size, H, W)，每个元素是类别索引
    
    返回:
        平均损失值
    
    实现细节：
    - cross_entropy: 计算每个像素的交叉熵损失
    - reduction='none': 不自动求平均，保留每个像素的损失
    - .mean(1).mean(1): 对高度和宽度维度求平均
    """
    return F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)


# ==================== 设置训练参数 ====================
num_epochs = 5              # 训练5个epoch
lr = 0.001                  # 学习率
wd = 1e-3                   # 权重衰减（L2正则化）
devices = d2l.try_all_gpus() # 尝试使用所有可用的GPU

# 创建优化器
# SGD: 随机梯度下降
# weight_decay: 权重衰减，防止过拟合
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)

# 开始训练
# train_ch13是d2l提供的训练函数，专门用于计算机视觉任务
# 它会：
# 1. 在每个epoch遍历训练数据
# 2. 计算损失并更新权重
# 3. 在测试集上评估性能
# 4. 绘制训练曲线
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

In [ ]:
# ==================== 预测和可视化 ====================

def predict(img):
    """
    对单张图像进行语义分割预测
    
    参数:
        img: 输入图像张量，形状(C, H, W)
    
    返回:
        pred: 预测的类别索引，形状(H, W)
    """
    # 归一化图像（使用与训练时相同的归一化）
    # unsqueeze(0): 添加批次维度 (C, H, W) -> (1, C, H, W)
    X = test_iter.dataset.normalize_image(img).unsqueeze(0)
    
    # 在GPU上进行预测
    # net(X): 得到形状(1, 21, H, W)的输出，每个通道是一个类别的分数
    # argmax(dim=1): 在类别维度上取最大值的索引，得到(1, H, W)
    pred = net(X.to(devices[0])).argmax(dim=1)
    
    # 移除批次维度，返回(H, W)的类别索引
    return pred.reshape(pred.shape[1], pred.shape[2])


def label2image(pred):
    """
    将类别索引转换为RGB彩色图像
    
    参数:
        pred: 类别索引张量，形状(H, W)
    
    返回:
        彩色分割图，形状(H, W, 3)
    """
    # VOC_COLORMAP是预定义的颜色映射表
    # 将其转换为张量并放到GPU上
    colormap = torch.tensor(d2l.VOC_COLORMAP, device=devices[0])
    
    # 确保pred是长整型（用作索引）
    X = pred.long()
    
    # 使用类别索引从颜色映射表中查找对应的RGB颜色
    # colormap[X, :] 会根据X中的类别索引返回对应的RGB值
    return colormap[X, :]


# ==================== 在测试集上进行预测和可视化 ====================

# 下载VOC数据集并获取路径
voc_dir = d2l.download_extract('voc2012', 'VOCdevkit/VOC2012')

# 读取测试集的图像和标注
test_images, test_labels = d2l.read_voc_images(voc_dir, False)

# 设置要显示的图像数量
n, imgs = 4, []

# 对前n张测试图像进行预测
for i in range(n):
    # 定义裁剪区域：从左上角(0,0)开始，裁剪320x480的区域
    crop_rect = (0, 0, 320, 480)
    
    # 裁剪原始图像
    X = torchvision.transforms.functional.crop(test_images[i], *crop_rect)
    
    # 预测并转换为彩色图像
    pred = label2image(predict(X))
    
    # 收集三张图像：原图、预测结果、真实标注
    imgs += [
        X.permute(1, 2, 0),      # 原始图像，调整为(H,W,C)格式
        pred.cpu(),               # 预测的彩色分割图，移到CPU
        torchvision.transforms.functional.crop(
            test_labels[i], *crop_rect).permute(1, 2, 0)  # 真实标注
    ]

# 显示结果：3行，每行n张图像
# 第一行：原始图像
# 第二行：模型预测结果
# 第三行：真实标注
# imgs[::3]: 每隔3个取一个，即原始图像
# imgs[1::3]: 预测结果
# imgs[2::3]: 真实标注
d2l.show_images(imgs[::3] + imgs[1::3] + imgs[2::3], 3, n, scale=2)